The CNN Money’s Market Movers website (https://money.cnn.com/data/hotstocks/ ) tracks the most active stocks on a real time basis. Specifically, the most active, the top gainers and top losers are listed at any instance in time. You will first write Python scripts that collect the list of most actives, gainers and losers from the above website. Next, your programs should take the ticker symbols of these companies and build a csv file (called stocks.csv) with data about each stock from the website: https://finance.yahoo.com/quote/AMD?p=AMD&.tsrc=fin-srch-v1 which gives the quote for ticker symbol AMD as an example. The data to be collected from the Yahoo Finance site should include:
> 1. OPEN price
2. PREV CLOSE price
3. VOLUME
4. MARKET CAP

Your code should also list the names of the companies in the order and categories listed in the website: https://money.cnn.com/data/hotstocks/ and ask the user to choose a company to get the data on. Once the user chooses the company of interest, your program should display its corresponding data (Open, Prev Close, Volume and Market Cap).

In [3]:
import threading # Importing the threading library...
import requests # Import request library...
from bs4 import BeautifulSoup # Importing the Beautiful Soap Library...
catagories = ["Most Actives", "Gainers","Losers"]
def get_top_stocks(): # Function to fetch the 3 catagory stocks and return them in a list. 
    stock_url = "https://money.cnn.com/data/hotstocks/" # The URL Used to fetch the data..
    requested_page = requests.get(stock_url) # Get the HTML for the requested URL
    soup = BeautifulSoup(requested_page.content, 'html5lib') # Convert it into Soup for easy handling
    tables = soup.findAll('table', attrs={'class':"wsod_dataTable wsod_dataTableBigAlt"}) # Look for the table with data.
    res_list = []
    for cat, stocks in zip(catagories, tables): # cat-> ["Most Actives", "Gainers","Losers"] [[Stocks],[[Stocks]],[[Stocks]]]
        print(cat+":") # print the catagory
        for entry in stocks.findAll('td'): # parse the stocks
            if entry.a!=None: # check of it's not none.. only print non-none stocks
                symbol = entry.a.text # Symbols and Names are extracted form "a" and "span" fields in stocks
                name = entry.span.text
                print(symbol,name) # Print them for the user 
                res_list.append([cat,symbol,name]) # store them to return as value
        print() # for new line
    return res_list # list of [Catagory, Symbol, Name] of all stocks
def get_stock_info_yahoo(stock): # Fetches the stock information, argument stock symbol.
    user_stock_url = "https://finance.yahoo.com/quote/{0}?p={0}&.tsrc=fin-srch-v1".format(stock) # Form the URL with the stock symbol.
    user_requested_page = requests.get(user_stock_url) # get the HTML
    soup = BeautifulSoup(user_requested_page.content, 'html5lib') # Convert it into Soup for easy parsing.
    tables = soup.findAll('div', attrs={'id':"quote-summary"}) # Look for the table of data
    table = (tables[0].findAll("table")) # get the sub-table with values.
    result = []
    for entry in table:
        allow_next = False
        for parameter in entry.findAll("td"): # Check if the parameter is as per our need, if so store the next value in list.
            if (parameter.text) in {"Previous Close", "Open", "Volume", "Market Cap"}:
                allow_next = True # Variable to signify that the next value is useful, store it. Assuming values will always come in the order "Previous Close", "Open", "Volume", "Market Cap"
            elif allow_next==True:
                allow_next=False # Check the next value while this value will be stored in the res
                result.append(parameter.text) # Adding the value in the list
    return result # returning the list of data to the user.. list of ["Previous Close", "Open", "Volume", "Market Cap"]

def write_in_file(stock_list): # Used to store the stock data in file.. for each stock call the ticker 
    import csv
    with open("stock.csv","w") as file: # Opening the csv file..
        file = csv.writer(file, delimiter=',', quotechar='"') # Setting up the writer
        file.writerow(["Catagory", "Symbol", "Name", "Previous Close", "Open", "Volume", "Market Cap"]) # Writing Headder
        for stock in stock_list:
            file.writerow(stock+get_stock_info_yahoo(stock[1])) # Calling the yahoo stock info and writing it in csv with catagory, symbol, name of stock

print("This is a program to scrape data from the https://money.cnn.com/data/hotstocks/ for a class project.")
print("Which stock are you interested in:")
res_list = get_top_stocks() # Get the stock catagory, symbol, name
th = threading.Thread(target=write_in_file,args=[res_list]) # Run the process in background as the process is slow.
th.start() # begin fetching price for each stock and store it in the file.
user_stock = input("User inputs:") # Ask user for input
for kk in res_list:
    if user_stock == kk[1]: # If the user did asked for a stock in the list
        print("The data for {} {} is the following:".format(user_stock, kk[2])) # print this message.
        break
if len(user_stock)==0: # Check if the len is graeter than 0
    print("None cannot be valid input, please enter the stock name.")
else:
    for Type,Value in zip(["Previous Close", "Open", "Volume", "Market Cap"], get_stock_info_yahoo(user_stock)):
        print("{} {}".format(Type,Value)) # Call the yahoo link with stock name and print the result
th.join() # Wait for the thread to finish writing in the file.... NOTE the star sign on this cell to see if process is finished.

This is a program to scrape data from the https://money.cnn.com/data/hotstocks/ for a class project.
Which stock are you interested in:
Most Actives:
GE General Electric Co
BAC Bank of America Corp
AMD Advanced Micro Devices Inc
F Ford Motor Co
FCX Freeport-McMoRan Inc
AAPL Apple Inc
INTC Intel Corp
BMY Bristol-Myers Squibb Co
WFC Wells Fargo & Co
MSFT Microsoft Corp

Gainers:
CCI Crown Castle International Corp
DXC DXC Technology Co
TTWO Take-Two Interactive Software Inc
SLG SL Green Realty Corp
NEM Newmont Goldcorp Corp
PEAK Healthpeak Properties Inc
EXR Extra Space Storage Inc
PSA Public Storage
NRG NRG Energy Inc
VNO Vornado Realty Trust

Losers:
APA Apache Corp
UNM Unum Group
XEC Cimarex Energy Co
FCX Freeport-McMoRan Inc
LB L Brands Inc
CPRI Capri Holdings Ltd
FDX FedEx Corp
LNC Lincoln National Corp
TPR Tapestry Inc
NOV National Oilwell Varco Inc

User inputs:APA
The data for APA Apache Corp is the following:
Previous Close 19.54
Open 19.25
Volume 15,089,803
Market Cap 6.912B
